<a href="https://colab.research.google.com/github/ignacioaranguren1/bd_2/blob/main/bd_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os

from tqdm import tqdm
from datetime import datetime
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from kerastuner.tuners import RandomSearch

import kerastuner

ModuleNotFoundError: No module named 'kerastuner'

In [8]:
%matplotlib inline

datapath = '/Users/ignacioaranguren/bd_2/data'
os.chdir(datapath)

1.In the data used by Gu, Kelly and Xiu (RFS 2019 – provided in class), use a similar procedure to theirs to predict stock returns with neural networks. Start by finding a suitable baseline configuration, and use a validation procedure to pick optimal hyperparameters for three neural network models: One with 2 hidden layers, one with 3 hidden layers, and one with 4 hidden layers. 

In [9]:
data = pd.read_pickle('returns_chars_panel.pkl')
macro = pd.read_pickle('macro_timeseries.pkl')

In [10]:
def train_validation_test_split(data,train_end_date,validation_end_date):
  tmp = data.reset_index()
  train = tmp[tmp.date<=train_end_date].set_index(['date','permno'],drop=True)
  validation = tmp[(tmp.date>train_end_date) & (tmp.date<=validation_end_date)].set_index(['date','permno'],drop=True)
  test = tmp[tmp.date>validation_end_date].set_index(['date','permno'],drop=True)
  return train,validation,test

In [11]:
data.head()

,date,permno,excess_ret,ret,rfree,mvel1,beta,betasq,chmom,dolvol,...,stdacc,stdcf,ms,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade
0,1986-02-01,10000,-0.262610,-0.257143,0.005467,-0.375440,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.572822,0.361538,0.938929,0.873640,0.399871,0.120000,-0.177810
1,1986-03-01,10000,0.360335,0.365385,0.005050,-0.496811,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.417504,0.520205,-0.138898,0.328870,0.078782,-0.411202,0.130393
2,1986-04-01,10000,-0.103717,-0.098592,0.005125,-0.401783,0.0,0.0,0.0,-0.521182,...,0.0,0.0,0.0,0.091488,0.329401,0.813365,0.660035,0.142035,-0.131985,0.119017
3,1986-05-01,10000,-0.227831,-0.222656,0.005175,-0.435735,0.0,0.0,0.0,-0.439391,...,0.0,0.0,0.0,0.329949,0.438295,-0.683852,-0.759048,0.388029,-0.199555,0.135402
4,1986-06-01,10000,-0.009883,-0.005025,0.004858,-0.534203,0.0,0.0,0.0,-0.499678,...,0.0,0.0,0.0,0.510140,0.610586,0.724257,0.608819,-0.128302,-0.025964,0.007879


In [12]:
data.columns

Index(['date', 'permno', 'excess_ret', 'ret', 'rfree', 'mvel1', 'beta',
       'betasq', 'chmom', 'dolvol', 'idiovol', 'indmom', 'mom1m', 'mom6m',
       'mom12m', 'mom36m', 'pricedelay', 'turn', 'absacc', 'acc', 'age', 'agr',
       'bm', 'bm_ia', 'cashdebt', 'cashpr', 'cfp', 'cfp_ia', 'chatoia',
       'chcsho', 'chempia', 'chinv', 'chpmia', 'convind', 'currat', 'depr',
       'divi', 'divo', 'dy', 'egr', 'ep', 'gma', 'grcapx', 'grltnoa', 'herf',
       'hire', 'invest', 'lev', 'lgr', 'mve_ia', 'operprof', 'orgcap',
       'pchcapx_ia', 'pchcurrat', 'pchdepr', 'pchgm_pchsale', 'pchquick',
       'pchsale_pchinvt', 'pchsale_pchrect', 'pchsale_pchxsga', 'pchsaleinv',
       'pctacc', 'ps', 'quick', 'rd', 'rd_mve', 'rd_sale', 'realestate',
       'roic', 'salecash', 'saleinv', 'salerec', 'secured', 'securedind',
       'sgr', 'sin', 'sp', 'tang', 'tb', 'aeavol', 'cash', 'chtx', 'cinvest',
       'ear', 'nincr', 'roaq', 'roavol', 'roeq', 'rsup', 'stdacc', 'stdcf',
       'ms', 'baspread'

In [13]:
data.iloc[:,5:].columns

Index(['mvel1', 'beta', 'betasq', 'chmom', 'dolvol', 'idiovol', 'indmom',
       'mom1m', 'mom6m', 'mom12m', 'mom36m', 'pricedelay', 'turn', 'absacc',
       'acc', 'age', 'agr', 'bm', 'bm_ia', 'cashdebt', 'cashpr', 'cfp',
       'cfp_ia', 'chatoia', 'chcsho', 'chempia', 'chinv', 'chpmia', 'convind',
       'currat', 'depr', 'divi', 'divo', 'dy', 'egr', 'ep', 'gma', 'grcapx',
       'grltnoa', 'herf', 'hire', 'invest', 'lev', 'lgr', 'mve_ia', 'operprof',
       'orgcap', 'pchcapx_ia', 'pchcurrat', 'pchdepr', 'pchgm_pchsale',
       'pchquick', 'pchsale_pchinvt', 'pchsale_pchrect', 'pchsale_pchxsga',
       'pchsaleinv', 'pctacc', 'ps', 'quick', 'rd', 'rd_mve', 'rd_sale',
       'realestate', 'roic', 'salecash', 'saleinv', 'salerec', 'secured',
       'securedind', 'sgr', 'sin', 'sp', 'tang', 'tb', 'aeavol', 'cash',
       'chtx', 'cinvest', 'ear', 'nincr', 'roaq', 'roavol', 'roeq', 'rsup',
       'stdacc', 'stdcf', 'ms', 'baspread', 'ill', 'maxret', 'retvol',
       'std_dolvol', 'std_

In [14]:
macro.head()

,date,dp,ep,b/m,crsp_spvw,svar,tbl,tms,dfy,dfr,ntis,infl
671,1926-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
672,1927-01-01,-2.973012,-2.386837,0.441476,0.026047,0.000465,0.0307,0.0047,0.0100,-0.0022,0.050876,0.000000
673,1927-02-01,-2.942374,-2.374773,0.443706,-0.002910,0.000470,0.0323,0.0028,0.0095,-0.0019,0.050824,-0.011299
674,1927-03-01,-2.979535,-2.430353,0.428501,0.045522,0.000287,0.0329,0.0018,0.0092,-0.0019,0.051668,-0.005714
675,1927-04-01,-2.976535,-2.445079,0.469765,0.007324,0.000924,0.0320,0.0011,0.0092,-0.0170,0.046357,-0.005747


In [15]:
data_merged = pd.merge(data,macro,on=['date'])
datelist = list(set(data_merged['date']))
datelist.sort()
data_merged.set_index(['date','permno'],drop=True,inplace=True)

In [ ]:
train_ratio = 0.5
validation_ratio = 0.25
train_date = datelist[int(len(datelist)*train_ratio)]
validation_date = datelist[int(len(datelist)*(train_ratio+validation_ratio))]#can also use cross validation
X = data_merged.iloc[:,5:].copy()
y = data_merged['excess_ret'].copy()

In [ ]:
X_train,X_validation,X_test = train_validation_test_split(X,train_date,validation_date)
y_train,y_validation,y_test = train_validation_test_split(y,train_date,validation_date)

In [ ]:
class HyperRegressor(kerastuner.HyperModel):
  def build(self, hp):
    model = Sequential()
    model.add(layers.Input(shape=(103,)))
    for i in range(n):
      model.add(layers.Dense(units=hp.Int(f'units_{i+1}',min_value=16,max_value=160,step=16),activation='relu'))
    model.add(layers.Dense(1, activation='linear'))# or linear
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='mse')
    model.add(layers.Dropout(rate=hp.Choice('dropout', values=[0.2,0.4,0.6,0.8])))
    return model

  def fit(self, hp, model, x, y, validation_data, **kwargs):
      model.fit(x, y, **kwargs)
      x_val, y_val = validation_data
      y_pred = model.predict(x_val)
      # Return a single float to minimize.
      return np.mean((y_pred - y_val)**2)

In [ ]:
def tune_model(n=2):
  tuner = RandomSearch(
      hypermodel=HyperRegressor(),
      # objective = 'mse',
      # Objective is the return value of `HyperModel.fit()`.
      max_trials=5,   #5 trails
      executions_per_trial=3, #test 3 times each trail
      overwrite=True,
      directory='my_dir',
      project_name=f'NN_new_{n}')
  tuner.search(X_train.values, y_train.values,epochs=3,validation_data=(X_validation.values, y_validation.values))
  return tuner

In [ ]:
models=[]
parameters=[]
tuners=[]
for n in range(2,5):
  tuner = tune_model(n)
  parameters.append(tuner.get_best_hyperparameters)
  models.append(tuner.get_best_models(1))
  tuners.append(tuner)
with open('parameters.pkl','wb') as f:
  pickle.dump(parameters,f,protocol=4)
with open('models.pkl','wb') as f:
  pickle.dump(parameters,f,protocol=4)
with open('tuners.pkl','wb') as f:
  pickle.dump(parameters,f,protocol=4)
  


Trial 3 Complete [00h 19m 12s]
default_objective: 0.0403921794318898

Best default_objective So Far: 0.040233786785545574
Total elapsed time: 00h 57m 44s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
64                |144               |units_1
96                |112               |units_2
64                |112               |units_3
0.001             |0.0001            |learning_rate
0.6               |0.8               |dropout

Epoch 1/3
39354/39354 [==============================] - 110s 3ms/step - loss: 0.0227
Epoch 2/3
39354/39354 [==============================] - 110s 3ms/step - loss: 0.0225
Epoch 3/3
39354/39354 [==============================] - 110s 3ms/step - loss: 0.0224
Epoch 1/3
39354/39354 [==============================] - 110s 3ms/step - loss: 0.0228
Epoch 2/3
39354/39354 [==============================] - 110s 3ms/step - loss: 0.0225
Epoch 3/3
39354/39354 [==============================] - 109s 3ms/step - loss: 0.0224
Epoch 1/3
207

In [ ]:
dir(tuner)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_and_fit_model',
 '_build_hypermodel',
 '_configure_tensorboard_dir',
 '_deepcopy_callbacks',
 '_display',
 '_get_checkpoint_fname',
 '_get_tensorboard_dir',
 '_get_tuner_fname',
 '_override_compile_args',
 '_populate_initial_space',
 '_save_n_checkpoints',
 '_try_build',
 'directory',
 'distribution_strategy',
 'executions_per_trial',
 'get_best_hyperparameters',
 'get_best_models',
 'get_state',
 'get_trial_dir',
 'hypermodel',
 'load_model',
 'logger',
 'loss',
 'max_model_size',
 'metrics',
 'on_batch_begin',
 'on_batch_end',
 'on_epoch_begin',
 'on_epoch_end',
 'on_search_begin',
 'on_search_end',
 'on_trial_b

In [ ]:
# with open('model_tuners.pkl','rb') as f:
#   models = pickle.load(f)

In [ ]:
models

[[<keras.engine.sequential.Sequential at 0x7f4cf843a710>,

In [ ]:
dir(model)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [ ]:
model = Sequential()
model.add(layers.Dense(16, input_dim=2, activation='relu'))
model.add(layers.Dense(16, input_dim=2, activation='relu'))
model.add(layers.Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='adam')
model.fit(X, y, epochs=16, batch_size=64,verbose=True)

In [ ]:
train_result = model.evaluate(X_train.values,y_train.values,batch_size=10000)
test_result = model.evaluate(X_test.values,y_test.values)
print('TRAIN MSE:',train_result)
print('TEST MSE:',test_result)

34491/34491 [==============================] - 80s 2ms/step - loss: 0.0259
TRAIN MSE: 0.022671449929475784
TEST MSE: 0.025928305462002754


In [ ]:
test_result

0.025928305462002754

2.Use test data to get an idea of the out of sample performance of each model. Convert the standard MSE metric for out of sample performance to the “R2 out of sample” metric that was discussed in class. Compare your results to those in Gu-Kelly-Xiu and comment on the differences. 

In [ ]:
# from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
# import eli5
# from eli5.sklearn import PermutationImportance

# def base_model():
#     model = Sequential()        
#     ...
#     return model

# X = ...
# y = ...

# my_model = KerasRegressor(build_fn=base_model, **sk_params)    
# my_model.fit(X,y)

# perm = PermutationImportance(my_model, random_state=1).fit(X,y)
# eli5.show_weights(perm, feature_names = X.columns.tolist())

3.Pick the model that performs the best out of sample, and interpret its output by doing the following analysis of variable importance:
a.	First, for all stock characteristics, get variable importance by setting one predictor at a time to zero and finding the decrease in out of sample R2. Show a table of the 10 most important variables according to this measure, and give an economic interpretation. 


b.	Second, get a measure of the joint importance of all our “macro predictors” (i.e., those taken from Welch and Goyal 2008), by setting them all to zero and finding the decrease in out of sample R2. Comment on how important macroeconomic variables are relative to stock characteristics in predicting returns. 

c.	Repeat the two steps above, but by using a measure of the sensitivity of predictions to each input variable, as outlined in the lectures.

4.Fit a penalised linear model (LASSO) to the same data, using validation data to pick the best penalty (e.g., you can use the “sklearn” package in Python to do this easily). Compare its test data performance to the neural network. 

5.Suppose somebody tells you to collect 10 more micro or macro variables that can predict returns and are not in our current dataset. How would you choose those variables, based on the intuitions you have gained in this project?